In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from sklearn.model_selection import train_test_split
from lib.reader import *
from lib.preprocess import *
import nltk
from sklearn.pipeline import *
from sklearn.feature_extraction.text import *
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import *
from functools import partial
from lib.morph import *

In [2]:
data_loc = "../data/id-syntax-sentence-classification.xlsx"
data: pd.DataFrame = load_as_dataframe(data_loc)

In [3]:
data.head()

,Kalimat,simple sentence,compound sentence,complex sentence,compound-complex sentence,incomplete sentence,transitive sentence,intransitive sentence,sentence in active voice,sentence in passive voice,Noun-predicate sentence,Adjective-predicate sentence,Prepositional phrase-predicate sentence,Numeral-predicate sentence,declarative sentence,interrogative sentence,imperative sentence,exclamative sentence,inverted sentence,sentences with dislocation
0,Saat ini ia menjabat sebagai Wakil Bupati Band...,True,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False
1,Sahrul mengawali kariernya dari dunia model.,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False
2,Kemudian merambah ke dunia tarik suara dan akt...,False,False,False,False,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False
3,Sinetron yang melambungkan namanya adalah Jin ...,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True
4,Setelah cukup lama membintangi sinetron terseb...,False,False,False,True,False,True,True,True,False,False,False,False,False,True,False,False,False,False,False


In [4]:
filtered_data = data[~(
    (data["declarative sentence"] & data["interrogative sentence"]) |
    (data["simple sentence"] & data["complex sentence"]) |
    (data["declarative sentence"] & data["imperative sentence"]) |
    (data["simple sentence"] & data["compound-complex sentence"]) |
    (data["simple sentence"] & data["compound sentence"])
)].copy()

In [5]:
filtered_data.loc[:, "sentences with dislocation"] = filtered_data.loc[:, "sentences with dislocation"].astype(bool)

In [6]:
y_columns = [col for col in data.columns if col != "Kalimat"]

In [58]:
filtered_data = filtered_data.reset_index()

In [93]:
X = filtered_data
y = filtered_data[y_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=32)

In [94]:
X_train = X_train.copy()
X_test = X_test.copy()

In [95]:
from concurrent.futures import ProcessPoolExecutor

In [96]:
def parse_with_index(i, text):
    return (i, parse(text))

In [97]:
process_pool = ProcessPoolExecutor(12)

In [65]:
futures = [
    process_pool.submit(parse_with_index, i, text) 
    for i, text in zip(X_train.index, X_train["Kalimat"].values)
]

In [66]:
for f in futures:
    f.result()

In [67]:
X_train = X_train.copy()

In [68]:
X_train["parsed"] = ""

In [69]:
for future in futures:
    i, parsed = future.result()
    X_train.loc[i, "parsed"] = parsed

In [70]:
X_train.to_csv("../data/train.csv")

In [72]:
idx = 84
print(X_train.iloc[idx]["Kalimat"])
print(X_train.iloc[idx]["parsed"])

Bagi sikerei yang berjenis kelamin perempuan, pada dasarnya adalah bersifat membantu sikerei laki-laki dan biasanya adalah istri dari sikerei tersebut.
^bagi<r>_R--$ ^sikerei<x>_X--$ ^yang<s>_S--$ ^ber+jenis<n>_VSA$ ^kelamin<n>_NSD$ ^perempuan<n>_NSD$ ^pada<b>_B--$ ^dasarnya<b>_B--$ ^adalah<o>_O--$ ^ber+sifat<n>_VSA$ ^meN+bantu<v>_VSA$ ^sikerei<x>_X--$ ^lakilaki<x>_X--$ ^dan<h>_H--$ ^biasanya<d>_D--$ ^adalah<o>_O--$ ^istri<n>_NSD$ ^dari<r>_R--$ ^sikerei<x>_X--$ ^tersebut<b>_B--$


In [99]:
test_futures = [
    process_pool.submit(parse_with_index, i, text) 
    for i, text in zip(X_test.index, X_test["Kalimat"].values)
]

In [100]:
for future in test_futures:
    i, parsed = future.result()
    X_test.loc[i, "parsed"] = parsed

In [101]:
X_test.to_csv("../data/test.csv")

In [85]:
X_test.loc[2843]

index                                                                                  352.0
Kalimat                                    Jempol terbalik karena pemerintah, lembaga yan...
simple sentence                                                                        False
compound sentence                                                                      False
complex sentence                                                                        True
compound-complex sentence                                                              False
incomplete sentence                                                                    False
transitive sentence                                                                     True
intransitive sentence                                                                   True
sentence in active voice                                                                True
sentence in passive voice                                             

In [89]:
X.shape

(9966, 21)

In [88]:
X_train.shape

(6976, 22)

In [92]:
X_test.shape

(9966, 22)